In [1]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
from utils.config import Config
import pandas as pd 
import numpy as np 
import mne
import seaborn as sns
from sklearn.pipeline import make_pipeline
sns.set(font_scale=1)
import matplotlib.pyplot as plt 
%matplotlib widget

from timeflux_rasr.estimation import RASR
from timeflux_blending.blending import Blending
from utils.viz import plotTimeSeries
from utils.utils import epoch
print("Config LOADED")

Config LOADED


# Tutorial Notebook for rASR implementation 
It consists in:
- loading the eeg signal and events from the *xdf* and *set* files (
- visualizing the power spectral density (check dead channels and line noise) 
- apply a sliding window on both training and test data
- train rASR on the training data
- apply rASR on test data with blending
- compare outcomes with different rASR parameters

## Set paths and loading data

In [2]:
training_files   = '/Users/louis/Dropbox/rASR Data/calibration data/sme_1_1.xdf_filt.set'
test_files       = '/Users/louis/Dropbox/rASR Data/filtered/sme_1_1.xdf_filt.set'
rasr_matlab_file = '/Users/louis/Dropbox/rASR Data/Riemannian ASR out/sme_1_1.set'

channels = ["Fp1", "Fp2", "T8", "Cz", "O1"]

mne_eeg_training = mne.io.read_raw_eeglab(training_files, preload=True).pick_channels(channels)
print(f"training data loaded with {len(mne_eeg_training.info['ch_names'])} channels and {mne_eeg_training.times[-1]-mne_eeg_training.times[0]:.1f} seconds")
mne_eeg_test     = mne.io.read_raw_eeglab(test_files, preload=True).pick_channels(channels)
print(f"test data loaded with {len(mne_eeg_test.info['ch_names'])} channels and {mne_eeg_test.times[-1]-mne_eeg_test.times[0]:.1f} seconds")
print("Files LOADED")
plt.close("all")
plot_args = dict(sfreq=mne_eeg_test.info["sfreq"], ch_names=mne_eeg_test.info["ch_names"], linewidth=0.25, scalings=1e-4)
plotTimeSeries(mne_eeg_test.get_data().T, color="black", **plot_args);

Reading /Users/louis/Dropbox/rASR Data/calibration data/sme_1_1.xdf_filt.fdt
Reading 0 ... 14999  =      0.000 ...    59.996 secs...
training data loaded with 5 channels and 60.0 seconds
Reading /Users/louis/Dropbox/rASR Data/filtered/sme_1_1.xdf_filt.fdt
Reading 0 ... 247499  =      0.000 ...   989.996 secs...
test data loaded with 5 channels and 990.0 seconds
Files LOADED


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Set RASR parameters, apply epoching and prepare pipelines

In [3]:
window_len        = 0.5  # in seconds
window_overlap    = 0.9  # in ratio of window_len
rejection_cutoff  = 3    # in std from the estimated distribution

# PREPARE TRAINING AND TEST EPOCHS
window_size     = int(mne_eeg_training.info["sfreq"] * window_len)  # epoch length in samples
window_interval = int(window_size * (1 - window_overlap))           # step interval in samples


# convert training data into epochs
X_training = np.swapaxes(epoch(mne_eeg_training.get_data(), window_size, window_interval, axis=-1), 1, 2)
print(f"training set with epochs of shape {X_training.shape}")

# convert test data into epochs
X_test = np.swapaxes(epoch(mne_eeg_test.get_data(), window_size, window_interval, axis=-1), 1, 2)
print(f"test set with epochs of shape {X_test.shape}")

rASR_pipeline     = make_pipeline(RASR(rejection_cutoff=rejection_cutoff))
blending_pipeline = make_pipeline(Blending(window_overlap=window_size-window_interval, merge=True))
print("Pipelines built")

training set with epochs of shape (1240, 125, 5)
test set with epochs of shape (20615, 125, 5)
Pipelines built


## Filter the epochs and reconstruct raw

In [4]:
import time
start = time.time()
rASR_pipeline.fit(X_training)
fitting_time = time.time()-start

start = time.time()
X_test_cleaned = rASR_pipeline.transform(X_test)
data_cleaned = blending_pipeline.fit_transform(X_test_cleaned)
transform_time = time.time()-start

print(f"data cleaned and reconstructed \n fit in {fitting_time*1e3 :.0f}ms\n transform in {transform_time*1e3 :.0f}ms ({transform_time*1e3/X_test.shape[0] :.2f}ms/epoch)")

data cleaned and reconstructed 
 fit in 657ms
 transform in 4759ms (0.23ms/epoch)


## Visualize comparison

In [5]:
# plot data with zoom effect
def plot_comparison(data, data_cleaned):
    plt.figure()

    ax2 = plt.subplot(212)
    plot_args["scalings"]=5e-5

    plotTimeSeries(data_cleaned, color="red", ax=ax2, **plot_args);
    plotTimeSeries(data, color="black", ax=ax2, **plot_args);
    
    ax1 = plt.subplot(211)

    plotTimeSeries(data_cleaned, color="red", ax=ax1, **plot_args);
    plotTimeSeries(data, color="black", ax=ax1, **plot_args);
    ax1.set_xlim(207,215)

    zoom_effect(ax1,ax2)
    # add a widget to scroll to the signal (update xlim)
    from ipywidgets import interact, FloatSlider
    def update_axis(xmin, xmax):
        if xmin<xmax:
            ax1.set_xlim(xmin,xmax)

    Tmax=data.shape[0] / mne_eeg_test.info["sfreq"]
    print(Tmax)
    i=FloatSlider(min=0, max=Tmax, step=10, continuous_update=False)
    ii=FloatSlider(min=0, max=Tmax, step=10, continuous_update=False)
    from ipywidgets import FloatSlider
    interact(update_axis,xmin=i, xmax=ii);

In [8]:
from utils.viz import zoom_effect
data = blending_pipeline.fit_transform(X_test)  # reconstruct test data for comparison (not cleaned)
plt.close("all")
plot_comparison(data, data_cleaned)
plt.title("before (black), RASR python (red)");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

989.972


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='xmin', max=989.972, step=10…

In [9]:

data_rasr_matlab     = mne.io.read_raw_eeglab(rasr_matlab_file, preload=True).pick_channels(channels).get_data()[:, :data_cleaned.shape[0]].T
print(data_rasr_matlab.shape)
plt.close("all")
plot_comparison(data_rasr_matlab, data_cleaned)
plt.title("RASR matlab (black), RASR python (red)");

Reading /Users/louis/Dropbox/rASR Data/Riemannian ASR out/sme_1_1.fdt
Reading 0 ... 247499  =      0.000 ...   989.996 secs...
(247493, 5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

989.972


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='xmin', max=989.972, step=10…